In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
high_MeetTijd = 2000
low_MeetTijd = 3
high_HR = 300
low_HR = 0
high_SpO2 = 100
low_SpO2 = 80
high_NIBP = 300
low_NIBP = 0

In [3]:
data = pd.read_csv('pdmssetw.csv')
data

,Unnamed: 0,MeetTijd,HR(bpm),SpO2(%),NIBP(mmHg),IcNaOk
0,0,[''],[''],[''],[''],0.0
1,1,[''],[''],[''],[''],0.0
2,2,[''],[''],[''],[''],0.0
3,3,[''],[''],[''],[''],0.0
4,4,[''],[''],[''],[''],0.0
...,...,...,...,...,...,...
45537,45537,"['19:14:19', '19:14:49', '19:15:19', '19:15:49...","['', '80', '81', '76', '83', '84', '79', '88',...","['100', '100', '100', '100', '100', '100', '10...","['', '123', '', '', '', '', '', '', '', '', ''...",0.0
45538,45538,"['15:17:54', '15:18:24', '15:18:54', '15:19:00...","['67', '', '68', '68', '68', '68', '71', '67',...","['97', '98', '98', '97', '95', '91', '98', '98...","['', '', '', '', '115', '', '', '', '', '113',...",0.0
45539,45539,"['17:39:54', '17:40:24', '17:40:54', '17:41:24...","['100', '103', '97', '101', '100', '106', '107...","['99', '99', '99', '99', '99', '99', '100', '9...","['', '', '', '', '', '', '', '', '', '', '', '...",0.0
45540,45540,"['19:24:26', '19:24:56', '19:25:25', '19:25:56...","['74', '75', '75', '74', '69', '73', '73', '72...","['98', '98', '96', '96', '97', '98', '98', '98...","['178', '', '', '', '', '', '', '185', '', '',...",1.0


In [4]:
data = data.drop(columns=['Unnamed: 0'])

In [5]:
np.nansum(data['IcNaOk'])/len(data)

0.08104606736638707

In [6]:
import ast
data.MeetTijd = data.MeetTijd.apply(ast.literal_eval)
data['HR(bpm)'] = data['HR(bpm)'].apply(ast.literal_eval).apply(lambda x: np.array(x))
data['SpO2(%)'] = data['SpO2(%)'].apply(ast.literal_eval).apply(lambda x: np.array(x))
data['NIBP(mmHg)'] = data['NIBP(mmHg)'].apply(ast.literal_eval).apply(lambda x: np.array(x))

In [7]:
data['HR(bpm)'] = data['HR(bpm)'].apply(lambda x: np.where(x=='', np.nan, x))
data['SpO2(%)'] = data['SpO2(%)'].apply(lambda x: np.where(x=='', np.nan, x))
data['NIBP(mmHg)'] = data['NIBP(mmHg)'].apply(lambda x: np.where(x=='', np.nan, x))

In [8]:
data = data.dropna().reset_index(drop=True)
data['HR(bpm)'] = data['HR(bpm)'].apply(lambda x: np.where((np.array(x,dtype=float)>high_HR) | (np.array(x,dtype=float)<low_HR),np.nan,x))
data['SpO2(%)'] = data['SpO2(%)'].apply(lambda x: np.where((np.array(x,dtype=float)>high_SpO2) | (np.array(x,dtype=float)<low_SpO2),np.nan,x))
data['NIBP(mmHg)'] = data['NIBP(mmHg)'].apply(lambda x: np.where((np.array(x,dtype=float)>high_NIBP) | (np.array(x,dtype=float)<low_NIBP),np.nan,x))

In [9]:
data['Length_MeetTijd'] = data['MeetTijd'].apply(lambda x: len(x))
data = data[(data['Length_MeetTijd']>3) & (data['Length_MeetTijd']<2000)]

In [10]:
from datetime import datetime

def time_diff(x):
    diff = (datetime.strptime(x[1],'%H:%M:%S')-datetime.strptime(x[0],'%H:%M:%S')).total_seconds()
    if (diff<0 or diff>90):
        diff = (datetime.strptime(x[2],'%H:%M:%S')-datetime.strptime(x[1],'%H:%M:%S')).total_seconds()
    return diff*len(x)

data['MeetTijd in seconds'] = data['MeetTijd'].apply(lambda x: time_diff(x))

In [11]:
data = data[(data['MeetTijd in seconds']>0) & (data['MeetTijd in seconds']<100000)]

In [12]:
data['Percentage HR nan'] = data['HR(bpm)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
data['Percentage SpO2 nan'] = data['SpO2(%)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
data['Percentage NIBP nan'] = data['NIBP(mmHg)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))

In [13]:
data90 = data[(data['Percentage HR nan']<0.10) & (data['Percentage SpO2 nan']<0.10) & (data['Percentage NIBP nan']<0.10)]
data95 = data[(data['Percentage HR nan']<0.05) & (data['Percentage SpO2 nan']<0.05) & (data['Percentage NIBP nan']<0.05)]
data99 = data[(data['Percentage HR nan']<0.01) & (data['Percentage SpO2 nan']<0.01) & (data['Percentage NIBP nan']<0.01)]
data100 = data[(data['Percentage HR nan']==0) & (data['Percentage SpO2 nan']==0) & (data['Percentage NIBP nan']==0)]

In [14]:
#pickle.dump(data, open("clean_data.pk", "wb"))
#pickle.dump(data100, open("data100.pk", "wb"))

In [15]:
data90['HR(bpm)'] = data90['HR(bpm)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))
data90['SpO2(%)'] = data90['SpO2(%)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))
data90['NIBP(mmHg)'] = data90['NIBP(mmHg)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))

<ipython-input-15-c1fb62223757>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data90['HR(bpm)'] = data90['HR(bpm)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))
<ipython-input-15-c1fb62223757>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data90['SpO2(%)'] = data90['SpO2(%)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))
<ipython-input-15-c1fb62223757>:3: SettingWithCopy

In [16]:
data90 = data90.drop(columns=['Percentage HR nan','Percentage SpO2 nan','Percentage NIBP nan','Length_MeetTijd'])

In [17]:
pickle.dump(data90, open("interpolated.pk", "wb"))